In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window 

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@olympicsprojectstorage.dfs.core.windows.net/athletes")

In [0]:
display(df)

In [0]:
df = df.fillna({"birth_place":"xyz","birth_country":"abc","residence_place":"unknown","residence_country":"aaa"})
df.display()

In [0]:
df_filtered = df.filter((col('current')==True) & (col('name').isin('GALSTYAN Slavik','HARUTYUNYAN Arsen','SEHEN Sajjad')))
df_filtered.display()

In [0]:
df = df.withColumn('height',col('height').cast(FloatType()))\
    .withColumn('weight',col('weight').cast(FloatType()))

df.display()

In [0]:
df_sorted = df.sort('height','weight',ascending=[0,1]).filter(col('weight')>0)
df_sorted.display()

In [0]:
df_sorted.withColumn('nationality',regexp_replace('nationality','United States', 'US'))
df_sorted.display()

In [0]:
df.groupBy('code').agg(count('code').alias('total_count')).filter(col('total_count')>1).display()

In [0]:
df_sorted = df_sorted.withColumnRenamed("code", "athlete_id")
df_sorted.display()


In [0]:
df_sorted = df_sorted.withColumn('occupation',split('occupation',','))
df_sorted.display()

In [0]:
df_sorted.columns

In [0]:
df_final = df_sorted.select('athlete_id',
 'current',
 'name',
 'name_short',
 'name_tv',
 'gender',
 'function',
 'country_code',
 'country',
 'country_long',
 'nationality',
 'nationality_long',
 'nationality_code',
 'height',
 'weight')

In [0]:
df_final.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_final.withColumn("cum_weight", sum("weight").over(Window.partitionBy("nationality").orderBy("height").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))).display()

In [0]:
df_final.write.format("delta")\
    .mode("append")\
    .option("path","abfss://silver@olympicsprojectstorage.dfs.core.windows.net/athletes")\
    .saveAsTable("olympicscatalog.silver.athletes")